In [1]:
import os
import shutil
import numpy as np


In [2]:
def get_extention(filename):
    return filename.split('.')[-1]
def get_sample_name(filename):
    return filename.split('_')[0]
    

In [3]:
DATASET = "SAM"

In [4]:
# Setup CONSTANTS
DATASETS_PATH = "/scratch/ottosson/datasets"
DATASET_PATH = os.path.join(DATASETS_PATH, DATASET)
DATASET_SMALL = DATASET + '_SMALL'
DATASET_SMALL_PATH = os.path.join(DATASETS_PATH, DATASET_SMALL)
if os.path.isdir(DATASET_SMALL_PATH):
    print("SMALL dir already exist, DO YOU WANT TO PRESIDE???")

# Reset dataset and create folder structure

In [5]:
SUB_FOLDERS = [['plantseg_training'],
                ['plantseg_training', 'train'],
                ['plantseg_training', 'val'],
                ['cellpose_training'],
                ['cellpose_training', 'train'],
                ['cellpose_training', 'val'],
                ['raw'],
                ['label'],
               ]

In [6]:
if os.path.isdir(DATASET_SMALL_PATH):
    shutil.rmtree(DATASET_SMALL_PATH)
os.mkdir(DATASET_SMALL_PATH)

for sub_branch in SUB_FOLDERS:
    os.mkdir(os.path.join(DATASET_SMALL_PATH,*sub_branch ))

# Chose the samples to be coppied

In [7]:
num_train_samples = 10
num_val_samples = 2

In [8]:
train_names = []
val_names = []
for f in os.listdir(os.path.join(DATASET_PATH, 'plantseg_training','train')):
    if get_extention(f) != 'h5': continue
    train_names.append(get_sample_name(f))
for f in os.listdir(os.path.join(DATASET_PATH, 'plantseg_training','val')):
    if get_extention(f) != 'h5': continue
    val_names.append(get_sample_name(f))

assert num_train_samples <= len(train_names), "Must be more samples then asked for!"
assert num_val_samples <= len(val_names), "Must be more samples then asked for!"

chosen_train_samples = list(np.random.choice(train_names, num_train_samples, replace = False))
chosen_val_samples = list(np.random.choice(val_names, num_val_samples, replace = False))
chosen_samples = chosen_train_samples + chosen_val_samples

In [9]:
print("Train: ", chosen_train_samples)
print("VAL: ", chosen_val_samples)
print("All: ", chosen_samples)

Train:  ['sample113', 'sample27', 'sample74', 'sample102', 'sample41', 'sample13', 'sample94', 'sample15', 'sample97', 'sample29']
VAL:  ['sample123', 'sample80']
All:  ['sample113', 'sample27', 'sample74', 'sample102', 'sample41', 'sample13', 'sample94', 'sample15', 'sample97', 'sample29', 'sample123', 'sample80']


# Copy over the chosen files

In [50]:
for sub_branch in SUB_FOLDERS:
    # Which files to copy
    if sub_branch[-1] == 'train':
        samples_to_copy = chosen_train_samples
    elif sub_branch[-1] == 'val':
        samples_to_copy = chosen_val_samples
    else:
        samples_to_copy = chosen_samples
    
    for filename in os.listdir(os.path.join(DATASET_PATH,*sub_branch)):
        if get_sample_name(filename=filename) not in samples_to_copy: continue
        shutil.copyfile(os.path.join(DATASET_PATH, *sub_branch, filename),
                        os.path.join(DATASET_SMALL_PATH, *sub_branch, filename))

# CAREFUL! This create the cellpose sub directories

In [51]:
# Change this path
cellpose_path = "/scratch/ottosson/datasets/Ovules/cellpose_training"
os.mkdir(os.path.join(cellpose_path ,'train'))

os.mkdir(os.path.join(cellpose_path ,'val'))

In [54]:
val_samples = ['N420ds2x', 'N464ds2x']
for f in os.listdir(cellpose_path):
    if get_extention(f) != 'tif': continue
    if get_sample_name(f) in val_samples:
        shutil.copyfile(os.path.join(cellpose_path,f),
                        os.path.join(cellpose_path,'val',f))
    else:
        shutil.copyfile(os.path.join(cellpose_path,f),
                        os.path.join(cellpose_path,'train',f))
    os.remove(os.path.join(cellpose_path,f))

In [53]:
# Make names
path = "/scratch/ottosson/datasets/Ovules/plantseg_training/val"
for f in os.listdir(path):
    if get_extention(f) != "h5": continue
    new_f = f.split('.')[0].replace('_','')+'_h5.h5'
    shutil.move(os.path.join(path, f), os.path.join(path,new_f ))